In [1]:

import requests
from bs4 import BeautifulSoup
from retry import retry
import time
import urllib.parse
import numpy as np
import pandas as pd

@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

def scrape_suumo(url, max_page):
    data_samples = []
    times = []

    start = time.time()

    for page in range(1, max_page + 1):
        before = time.time()

        soup = load_page(url.format(page))
        building_list = soup.find_all(class_='cassetteitem')

        for building in building_list:
            building_data = extract_building_data(building)
            data_samples += extract_room_data(building, building_data, url)

        time.sleep(1)
        after = time.time()

        print_progress(page, max_page, before, after, data_samples, times)

    finish = time.time()
    print('総経過時間：', finish - start)

    return data_samples

def extract_building_data(building):
    building_info = []
    building_info.append(building.find(class_='ui-pct ui-pct--util1').text)
    building_info.append(building.find(class_='cassetteitem_content-title').text)
    building_info.append(building.find(class_='cassetteitem_detail-col1').text)

    details = building.find(class_='cassetteitem_detail-col2')
    for detail in details.find_all(class_='cassetteitem_detail-text'):
        building_info.append(detail.text)

    details = building.find(class_='cassetteitem_detail-col3')
    for detail in details.find_all('div'):
        building_info.append(detail.text)

    return building_info

def extract_room_data(building, building_info, url):
    room_data_samples = []
    rooms = building.find(class_='cassetteitem_other')

    for room in rooms.find_all(class_='js-cassette_link'):
        room_data = extract_single_room_data(room, url)
        room_data_samples.append(building_info + room_data)

    return room_data_samples

def extract_single_room_data(room, url):
    room_info = []

    for id_, detail in enumerate(room.find_all('td')):
        if id_ == 2:
            room_info.append(detail.text.strip())
        elif id_ == 3:
            room_info.append(detail.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
            room_info.append(detail.find(class_='cassetteitem_price cassetteitem_price--administration').text)
        elif id_ == 4:
            room_info.append(detail.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
            room_info.append(detail.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
        elif id_ == 5:
            room_info.append(detail.find(class_='cassetteitem_madori').text)
            room_info.append(detail.find(class_='cassetteitem_menseki').text)
        elif id_ == 8:
            get_url = detail.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
            abs_url = urllib.parse.urljoin(url, get_url)
            room_info.append(abs_url)

    return room_info

def print_progress(page, max_page, before, after, data_samples, times):
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    print(f'総取得件数：{len(data_samples)}')
    complete_ratio = round(page / max_page * 100, 3)
    print(f'完了：{complete_ratio}%')
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    print(f'残り時間：{int(running_required_time / 3600)}時間{int((running_required_time % 3600) / 60)}分{int(running_required_time % 60)}秒\n')


# スクレイピング実行
url = 'https://suumo.jp/chintai/kanagawa/sc_yokohamashitsurumi/?page={}'
max_page = 155
data_samples = scrape_suumo(url, max_page)

# Pandas DataFrameに変換
correct_column_names = ['カテゴリ', '建物名', '住所', 'アクセス1', 'アクセス2', 'アクセス3', '築年数', '構造', '階数', '家賃', '管理費', '敷金', '礼金', '間取り', '面積', 'URL']
df = pd.DataFrame(data_samples, columns=correct_column_names)


#####ここまでスクレイピング
#####ここからクレンジング
# アクセス情報の処理と挿入
access_columns = ['アクセス1', 'アクセス2', 'アクセス3']
for col in access_columns:
    index = df.columns.get_loc(col)
    df[col + '_路線名'] = df[col].str.split('/', expand=True)[0]
    df[col + '_駅名'] = df[col].str.split('/', expand=True)[1].str.split(' 歩', expand=True)[0]
    df[col + '_徒歩分数'] = df[col].str.split('/', expand=True)[1].str.split(' 歩', expand=True)[1].str.replace('分', '', regex=True)
    df[col + '_徒歩分数'] = pd.to_numeric(df[col + '_徒歩分数'], errors='coerce')
    df.drop(col, axis=1, inplace=True)
    df.insert(index, col + '_路線名', df.pop(col + '_路線名'))
    df.insert(index + 1, col + '_駅名', df.pop(col + '_駅名'))
    df.insert(index + 2, col + '_徒歩分数', df.pop(col + '_徒歩分数'))


# 「家賃」、「敷金」、「礼金」の「-」を「0」に置換し、不要な文字列を除去して数値に変換
columns_to_convert = ['家賃', '敷金', '礼金']
for column in columns_to_convert:
    df[column] = df[column].replace('-', '0')
    df[column] = df[column].replace('万円', '', regex=True)
    df[column] = pd.to_numeric(df[column], errors='coerce') * 10000


# 「管理費」列の「-」を「0」に置換し、不要な文字列を除去して数値に変換
df['管理費'] = df['管理費'].replace('-', '0')
df['管理費'] = df['管理費'].replace('円', '', regex=True)
df['管理費'] = pd.to_numeric(df['管理費'], errors='coerce')


# 「面積」列の単位を除去し、数値に変換
df['面積'] = df['面積'].replace('m2', '', regex=True)
df['面積'] = pd.to_numeric(df['面積'], errors='coerce')



# 「築年数」列の処理
df['築年数'] = df['築年数'].replace('新築', '0')  # 「新築」を「0」に置き換え
df['築年数'] = df['築年数'].str.extract('(\d+)').astype(float)  # 数字の抽出と数値化




# すべての列が完全に一致する行の重複を削除
df = df.drop_duplicates()









1ページ目：2.4162700176239014秒
総取得件数：50
完了：0.645%
残り時間：0時間6分12秒

2ページ目：2.2621660232543945秒
総取得件数：86
完了：1.29%
残り時間：0時間5分57秒

3ページ目：2.344654083251953秒
総取得件数：180
完了：1.935%
残り時間：0時間5分55秒

4ページ目：2.043109893798828秒
総取得件数：274
完了：2.581%
残り時間：0時間5分42秒

5ページ目：1.7077760696411133秒
総取得件数：313
完了：3.226%
残り時間：0時間5分23秒

6ページ目：1.8562531471252441秒
総取得件数：358
完了：3.871%
残り時間：0時間5分13秒

7ページ目：1.7844841480255127秒
総取得件数：398
完了：4.516%
残り時間：0時間5分4秒

8ページ目：1.942406177520752秒
総取得件数：454
完了：5.161%
残り時間：0時間5分0秒

9ページ目：1.921475887298584秒
総取得件数：510
完了：5.806%
残り時間：0時間4分56秒

10ページ目：1.7686030864715576秒
総取得件数：549
完了：6.452%
残り時間：0時間4分50秒

11ページ目：1.7863118648529053秒
総取得件数：607
完了：7.097%
残り時間：0時間4分45秒

12ページ目：2.222113847732544秒
総取得件数：672
完了：7.742%
残り時間：0時間4分46秒

13ページ目：2.299595832824707秒
総取得件数：720
完了：8.387%
残り時間：0時間4分47秒

14ページ目：1.7778820991516113秒
総取得件数：767
完了：9.032%
残り時間：0時間4分43秒

15ページ目：1.752959966659546秒
総取得件数：818
完了：9.677%
残り時間：0時間4分38秒

16ページ目：2.106671094894409秒
総取得件数：863
完了：10.323%
残り時間：0時間4分37秒

17ページ目：1.8712718486785889秒
総取

In [2]:
print(df.columns)


Index(['カテゴリ', '建物名', '住所', 'アクセス1_路線名', 'アクセス1_駅名', 'アクセス1_徒歩分数', 'アクセス2_路線名',
       'アクセス2_駅名', 'アクセス2_徒歩分数', 'アクセス3_路線名', 'アクセス3_駅名', 'アクセス3_徒歩分数', '築年数',
       '構造', '階数', '家賃', '管理費', '敷金', '礼金', '間取り', '面積', 'URL'],
      dtype='object')


In [3]:
# 処理後のデータフレームの先頭部分を表示して確認
print(df.head())

      カテゴリ                  建物名               住所 アクセス1_路線名 アクセス1_駅名  \
0   賃貸アパート          Puerta Rojo  神奈川県横浜市鶴見区市場西中町      京急本線    鶴見市場駅   
1   賃貸アパート  京急本線 鶴見市場駅 2階建 築16年  神奈川県横浜市鶴見区市場西中町      京急本線    鶴見市場駅   
2  賃貸マンション             IN　BLOOM   神奈川県横浜市鶴見区潮田町４     ＪＲ鶴見線      浅野駅   
3  賃貸マンション    ＪＲ鶴見線 浅野駅 4階建 築8年   神奈川県横浜市鶴見区潮田町４     ＪＲ鶴見線      浅野駅   
4   賃貸アパート          カーサフェリーチェ矢向    神奈川県横浜市鶴見区矢向３     ＪＲ南武線      尻手駅   

   アクセス1_徒歩分数 アクセス2_路線名 アクセス2_駅名  アクセス2_徒歩分数 アクセス3_路線名  ...   築年数   構造  階数  \
0           7      京急本線     八丁畷駅        14.0   ＪＲ京浜東北線  ...  16.0  2階建  1階   
1           6     ＪＲ南武線     八丁畷駅        12.0   ＪＲ京浜東北線  ...  16.0  2階建  1階   
2           9      京急本線    京急鶴見駅        21.0     ＪＲ鶴見線  ...   8.0  4階建  3階   
3          10      京急本線    京急鶴見駅        22.0   ＪＲ京浜東北線  ...   8.0  4階建  3階   
4           5   ＪＲ京浜東北線      川崎駅        20.0   ＪＲ京浜東北線  ...   5.0  3階建  2階   

        家賃   管理費       敷金        礼金   間取り     面積  \
0  92000.0  3000  92000.0   92000.0  1LDK  33.85   


In [ ]:
#スプシ

In [5]:
pip install gspread oauth2client

  Obtaining dependency information for gspread from https://files.pythonhosted.org/packages/79/14/baddb424ffabc843d8b4776fbe4f840037fab100aa4e4a5e8cf5fbcafc8e/gspread-5.12.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 567.6 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for google-auth>=1.12.0 from https://files.pythonhosted.org/packages/f4/d2/9f6f3b9c0fd486617816cff42e856afea079d0bad99f0e60dc186c76b881/google_auth-2.25.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib>=0.4.1 from https://files.pythonhosted.org/packages/71/bf/9e125754d1adb3bc4bd206c4e5df756513b1d23675ac06caa471278d1f3f/google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Obtaining dependency information for pyasn1>=0.1.7 from https://files.pythonhosted.org/packages/d1/75/4686d2872bf2fc0b37917cbc8bbf0dd3a5cdb0990799be1b9cbf1e1eb733/pyasn1-0.5.1-py2.py3-none-any.whl.

In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def save_to_google_sheets(df, spreadsheet_name, worksheet_name):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('クレデンシャルファイル.json', scope)
    client = gspread.authorize(creds)

    try:
        # スプレッドシートを開くか、存在しなければ新規作成
        sheet = client.open(spreadsheet_name).sheet1
    except gspread.SpreadsheetNotFound:
        sheet = client.create(spreadsheet_name).sheet1

    # ワークシートの名前を変更
    sheet.update_title(worksheet_name)

    # DataFrame内のNaN値を0に置換
    df.fillna(0, inplace=True)

    # DataFrame を Google スプレッドシートに書き込む
    # 名前付き引数を使用して更新されたメソッドの使用法に従って修正
    sheet.update(range_name='A1', values=[df.columns.values.tolist()] + df.values.tolist())

# スクレイピングしたデータをスプレッドシートに保存
# dfはあなたのDataFrame変数です。適切なDataFrameを渡してください。
save_to_google_sheets(df, 'suumo1221', 'Sheet1')


/var/folders/xt/62y82p1s2f19jxr5_mt6x3900000gn/T/ipykernel_27021/1701189344.py:24: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update(range_name='A1', values=[df.columns.values.tolist()] + df.values.tolist())


In [7]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/62/1b/3ff6bd5f33c1a1780835725014ac480128d2d1e3244b2809275d0fa4f726/google_api_python_client-2.111.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0.dev0,>=1.31.5 from https://files.pythonhosted.org/packages/d6/c9/0462f037b62796fbda4801be62d0ae3147eaeb99e2939661580c98abe3eb/google_api_core-2.15.0-py3-none-any.whl.metadata
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Obtaining dependency information for googleapis-common-protos<2.0.dev0,>=1.56.2 from https://files.pythonhosted.org/packages/f0/43/c9d8f75ddf08e2a0a27db243c13a700c3cc7ec615b545b697cf6f715ad92/googleap

In [8]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

def list_google_drive_files(service_account_file):
    # サービスアカウント認証情報の読み込み
    creds = Credentials.from_service_account_file(
        service_account_file,
        scopes=['https://www.googleapis.com/auth/drive']
    )

    service = build('drive', 'v3', credentials=creds)

    # Googleドライブ内のファイルリストを取得
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)"
    ).execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(f"{item['name']} ({item['id']})")

# サービスアカウントのJSONキーファイルを指定
service_account_file = 'クレデンシャルファイル.json'

list_google_drive_files(service_account_file)


Files:
suumo1221 (1aAEDZjipNqKu5f3JbnPAnNCtK0GdSCgAUYMgYlyioas)
suumo12210045 (16LWcb9cE9iFVuNADWHmYf23ZyhwfqdkUgCqNiiflF-Q)
suumo12202145 (1v3jjF6Kn_G5kZtRAGSGXxuZFvMF--jdLBGjUbshYCro)
My Spreadsheet (1U8t4PPv9aMKtvIN3EfwrJMEKqstBo_ivVAdR1_2QVUo)
suumo_cleansing.csv (1dJU0ogxUHMYqYlIsoycqms9srmaLjTI8)


In [9]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import gspread

# サービスアカウントのクレデンシャルファイルのパスを設定
credentials_file_path = 'クレデンシャルファイル.json'

# クレデンシャルをロード
creds = Credentials.from_service_account_file(credentials_file_path)
scoped_creds = creds.with_scopes(['https://www.googleapis.com/auth/drive'])

# gspread を使用して認証
gc = gspread.authorize(scoped_creds)

# スプレッドシートを開く
spreadsheet_name = 'suumo1221'
spreadsheet = gc.open(spreadsheet_name)

# Google Drive サービスを取得
drive_service = build('drive', 'v3', credentials=scoped_creds)

# スプレッドシートのファイルIDを取得
spreadsheet_id = spreadsheet.id

# 「URLを知っている人はだれでも見れる」設定を適用
drive_service.permissions().create(
    fileId=spreadsheet_id,
    body={'type': 'anyone', 'role': 'reader'},
    fields='id'
).execute()

# スプレッドシートのウェブビューリンクを取得
web_view_link = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit"

print(web_view_link)


https://docs.google.com/spreadsheets/d/1aAEDZjipNqKu5f3JbnPAnNCtK0GdSCgAUYMgYlyioas/edit


In [10]:
#スプレッドシートを呼び出し格納

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

SP_CREDENTIAL_FILE = 'クレデンシャルファイル.json'
SP_COPE = [
    'https://www.googleapis.com/auth/drive',
    'https://spreadsheets.google.com/feeds'
]
SP_SHEET_KEY = '1aAEDZjipNqKu5f3JbnPAnNCtK0GdSCgAUYMgYlyioas'
SP_SHEET = 'Sheet1'

credentials = ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE, SP_COPE)
gc = gspread.authorize(credentials)

sh = gc.open_by_key(SP_SHEET_KEY)
worksheet = sh.worksheet(SP_SHEET)
data = worksheet.get_all_values()
print(data)


[['カテゴリ', '建物名', '住所', 'アクセス1_路線名', 'アクセス1_駅名', 'アクセス1_徒歩分数', 'アクセス2_路線名', 'アクセス2_駅名', 'アクセス2_徒歩分数', 'アクセス3_路線名', 'アクセス3_駅名', 'アクセス3_徒歩分数', '築年数', '構造', '階数', '家賃', '管理費', '敷金', '礼金', '間取り', '面積', 'URL'], ['賃貸アパート', 'Puerta Rojo', '神奈川県横浜市鶴見区市場西中町', '京急本線', '鶴見市場駅', '7', '京急本線', '八丁畷駅', '14', 'ＪＲ京浜東北線', '鶴見駅', '17', '16', '2階建', '1階', '92000', '3000', '92000', '92000', '1LDK', '33.85', 'https://suumo.jp/chintai/jnc_000071012556/?bc=100358009319'], ['賃貸アパート', '京急本線 鶴見市場駅 2階建 築16年', '神奈川県横浜市鶴見区市場西中町', '京急本線', '鶴見市場駅', '6', 'ＪＲ南武線', '八丁畷駅', '12', 'ＪＲ京浜東北線', '鶴見駅', '18', '16', '2階建', '1階', '92000', '3000', '92000', '92000', '1LDK', '35', 'https://suumo.jp/chintai/jnc_000071066695/?bc=100358119591'], ['賃貸マンション', 'IN\u3000BLOOM', '神奈川県横浜市鶴見区潮田町４', 'ＪＲ鶴見線', '浅野駅', '9', '京急本線', '京急鶴見駅', '21', 'ＪＲ鶴見線', '安善駅', '12', '8', '4階建', '3階', '94000', '3500', '94000', '141000', '1LDK', '40.72', 'https://suumo.jp/chintai/jnc_000047219877/?bc=100358012395'], ['賃貸マンション', 'ＪＲ鶴見線 浅野駅 4階建 築8年', '神奈川県横浜市鶴見区潮田町４

In [12]:
import pandas as pd

# データをDataFrameに変換
df = pd.DataFrame(data)

# CSVファイルに保存
df.to_csv('suumo1221.csv', index=False, encoding='utf-8-sig')
